Import libraries

In [0]:
import numpy as np
import os
import tensorflow as tf
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Noise = 0.1

Import Noisy Images

In [0]:
bad_frames = '/content/drive/My Drive/natural-images/natural_images/noisy_flower/0.1'

In [3]:
noisy_frames = []
for file in tqdm(sorted(os.listdir(bad_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(bad_frames + '/' + file, target_size=(420,540))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32')/255
    noisy_frames.append(image)

noisy_frames = np.array(noisy_frames)

In [4]:
print(noisy_frames.shape)

(843, 420, 540, 3)


Import Clean Images

In [0]:
good_frames = '/content/drive/My Drive/natural-images/natural_images/flower'

In [6]:
clean_frames = []
for file in tqdm(sorted(os.listdir(good_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(good_frames + '/' + file, target_size=(420,540))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32')/255
    clean_frames.append(image)

clean_frames = np.array(clean_frames)

In [7]:
print(clean_frames.shape)

(843, 420, 540, 3)


Split Images Into Training & Test

In [8]:
round(len(noisy_frames)*0.8)

674

In [0]:
#80% of images to training set
noisy_train = noisy_frames[0:round(len(noisy_frames)*0.8)]
noisy_test = noisy_frames[round(len(noisy_frames)*0.8):]

clean_train = clean_frames[0:round(len(clean_frames)*0.8)]
clean_test = clean_frames[round(len(clean_frames)*0.8):]

In [14]:
clean_frames.shape[1:4]

(420, 540, 3)

Autoencoder

In [0]:
autoencoder = tf.keras.models.Sequential()
autoencoder.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'same', input_shape = clean_frames.shape[1:4]))
autoencoder.add(tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same'))
autoencoder.add(tf.keras.layers.Conv2D(filters = 8, kernel_size = (3,3), activation = 'relu', padding = 'same'))
autoencoder.add(tf.keras.layers.Conv2D(filters = 8, kernel_size = (3,3), activation = 'relu', padding = 'same'))
autoencoder.add(tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same'))
autoencoder.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'same'))

In [18]:
autoencoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 420, 540, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 420, 540, 16)      4624      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 420, 540, 8)       1160      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 420, 540, 8)       584       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 420, 540, 16)      1168      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 420, 540, 32)      4640      
Total params: 13,072
Trainable params: 13,072
Non-trainable params: 0
__________________________________________________

In [0]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [20]:
history = autoencoder.fit(noisy_train, clean_train, epochs=5)

Epoch 1/5


ValueError: ignored